In [1]:
import math
import tensorflow as tf
import numpy as np
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import os
from tensorflow.contrib import rnn


tf.reset_default_graph()
tf.set_random_seed(777)

if "DISPLAY" not in os.environ:
    mpl.use('TkAgg')

import matplotlib.pyplot as plt

def MinMaxScaler(data):
    numerator = data - np.min(data,0)
    denominator = np.max(data,0)-np.min(data,0)
    return numerator / (denominator+1e-7)

xy = np.genfromtxt('/Users/yeseo/Desktop/City_Counted_TaxiMach_Link_Dataset_Full_201501 - 12.txt',delimiter = ',',dtype = None)

train_size = int(len(xy)*0.7)

xy= xy[:,:27]

test_set = xy[train_size:]

a = xy[:,:2]
b = xy[:,2:]
data_max = np.max(b,1)
data_min = np.min(b,1)
b= MinMaxScaler(b)
xy = np.hstack((a,b))

seq_length =6
data_dim =27
hidden_dim = 27
output_dim = 25
learning_rate = 0.1
iterations = 4001

train_set = xy[:train_size]


def build_dataset(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range (0,len(time_series)-seq_length):
        _x = time_series[i:i + seq_length, :]
        _y = time_series[i+seq_length,2:]
     
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

#train_set, test_set 만들기
trainX, trainY = build_dataset(train_set,seq_length)
testX,testY = build_dataset(test_set, seq_length)



X1 = tf.placeholder(tf.float32,[None, seq_length,data_dim])
Y1 = tf.placeholder(tf.float32,[None,25])

#LSTM CELL만들기
with tf.variable_scope("rnn1"):
    cell1 = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple = True, activation=tf.tanh)
    multi_cell1 = tf.contrib.rnn.MultiRNNCell([cell1]*5)
    outputs1,_states1 = tf.nn.dynamic_rnn(multi_cell1,X1,dtype = tf.float32)
    Y_pred = tf.contrib.layers.fully_connected(outputs1[:,-1], output_dim,activation_fn = None)

    loss1 =tf.reduce_mean(tf.square(Y_pred-Y1))
    train1 = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss1)

x1 = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25])*2
x2 = x1+0.33*2
x3 = x2+0.33*2

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(iterations):
        _, step_loss1 = sess.run([train1,loss1],feed_dict={X1:trainX, Y1:trainY})
        if i%500 == 0:
            print("[step: {}] loss: {}".format(i,step_loss1))

    
    #7th value from initial data
    test_predict = sess.run(Y_pred, feed_dict = {X1:[testX[0,:,:]]})
    
    #initial data shape = 6,27
    initial_data = testX[0,:,:]
    
    
    #day&time for 7th data
    if initial_data[5,1] != 47:
        day_time = np.array([[initial_data[5,0],(initial_data[5,1]+1)]])
    else:
        if initial_data[5,0] == 7:
            day_time = np.array([[1,0]])
        else:
            day_time = np.array([[(initial_data[5,0]+1),0]])
            
    #make 7th data
    seventh_data = np.hstack((day_time,test_predict))
    
    #1~5 data + 7th data
    new_testX = np.vstack((initial_data[1:,:],seventh_data))
        
    for j in range(len(testX)-1):
        #8th value from second data
        temp = sess.run(Y_pred,feed_dict = {X1:[new_testX]})
        
        test_predict = np.vstack((test_predict,temp))
        
        if new_testX[5,1] != 47:
            day_time = np.array([[new_testX[5,0],(new_testX[5,1]+1)]])
        else:
            if new_testX[5,0] == 7:
                day_time = np.array([[1,0]])
            else:
                day_time = np.array([[(new_testX[5,0]+1),0]])
                
        seventh_data = np.hstack((day_time,test_predict[[-1],:]))
        new_testX = np.vstack((new_testX[1:,:],seventh_data))
        
        
        
        


Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
[step: 0] loss: 0.17290590703487396
[step: 500] loss: 0.004770060535520315
[step: 1000] loss: 0.0010872113052755594
[step: 1500] loss: 0.0007264502346515656
[step: 2000] loss: 0.0005922699347138405
[step: 2500] loss: 0.0004762282769661397
[step: 3000] loss: 0.00042068600305356085
[step: 3500] loss: 0.0003649230638984591
[step: 4000] loss: 0.0003050308732781559


In [5]:
np.shape(day_time), np.shape(test_predict),np.shape(testY)

((1, 2), (1204, 25), (1204, 25))

In [3]:
def ReturnToOriginal(data,data_max,data_min):
    #og = data*(data_max+1e-7)-data_min*(data-1)
    data_max = data_max+(1e-7)
    for i in range(0,len(data)):
        data[i,:]= (data[i,:]*data_max[i])-(data[i,:]-1)*data_min[i]
    return data

test_predict_to_original = ReturnToOriginal(test_predict,data_max[train_size+seq_length:],data_min[train_size+seq_length:])
#testY_to_original = ReturnToOriginal(testY,data_max[train_size+seq_length:],data_min[train_size+seq_length:])

In [4]:
test_predict

array([[ 8712.184 ,  9656.146 , 10797.237 , ...,  9323.9375, 10269.972 ,
        10145.069 ],
       [ 9058.447 ,  9569.863 , 11435.064 , ...,  9571.143 , 10439.71  ,
        10182.795 ],
       [ 8528.131 ,  9307.904 , 10505.237 , ...,  8917.766 ,  9866.424 ,
         9502.258 ],
       ...,
       [11450.856 , 11078.237 , 15230.325 , ..., 13166.075 , 14382.214 ,
        14358.66  ],
       [12580.159 , 11698.277 , 16795.379 , ..., 14151.662 , 15643.353 ,
        15832.42  ],
       [12176.481 , 11069.055 , 16259.075 , ..., 13440.471 , 15015.209 ,
        15339.26  ]], dtype=float32)

In [3]:
testY

array([[ 8384, 10798, 10238, ..., 16611, 13497, 10030],
       [ 7941,  9716,  9503, ..., 15653, 12989,  9823],
       [ 9326, 10868,  9928, ..., 16364, 13762, 10774],
       ...,
       [ 9041,  9908, 12287, ..., 16500, 12015,  9102],
       [ 9569, 10277, 13272, ..., 17977, 13299, 10179],
       [ 9594,  9757, 13118, ..., 17758, 13227, 10024]])

In [5]:
test_predict_to_original

array([[ 8712.184 ,  9656.146 , 10797.237 , ...,  9323.9375, 10269.972 ,
        10145.069 ],
       [ 9058.447 ,  9569.863 , 11435.064 , ...,  9571.143 , 10439.71  ,
        10182.795 ],
       [ 8528.131 ,  9307.904 , 10505.237 , ...,  8917.766 ,  9866.424 ,
         9502.258 ],
       ...,
       [11450.856 , 11078.237 , 15230.325 , ..., 13166.075 , 14382.214 ,
        14358.66  ],
       [12580.159 , 11698.277 , 16795.379 , ..., 14151.662 , 15643.353 ,
        15832.42  ],
       [12176.481 , 11069.055 , 16259.075 , ..., 13440.471 , 15015.209 ,
        15339.26  ]], dtype=float32)

In [6]:
acc = (np.sum(np.square(test_predict-testY)))/(np.shape(test_predict)[0]*np.shape(test_predict)[1])

In [7]:
np.sqrt(acc)

2477.044988654444